In [462]:
import pandas as pd
import numpy as np
import csv
import math as m

In [463]:
# Converts WALS coordinate data n to nearest n.25 or n.75 in order to search for classification
def rco(num):
    n = num-int(num)
    if n<=0.5:
        return (int(num)+ 0.25)
    else:
        return(int(num)+0.75)

In [464]:
#Auxilary binary search function
def binary(alist, item):
    if len(alist) == 0:
        return (-1)
    else:
        midpoint = len(alist)//2
        if alist[midpoint]==item:
          return (midpoint)
        else:
          if item<alist[midpoint]:
            return binary(alist[:midpoint],item)
          else:
            return binary(alist[midpoint+1:],item)

In [465]:
#Spent a lot of the time finding the encoding
lang_df = pd.read_csv('language.csv', encoding = 'ISO-8859-1')

In [466]:
#koppen_1901-2010.tsv contains a file with latitude and longitudes that are close to .25 and .75 that gives the climate
koppen = np.genfromtxt("koppen_1901-2010.tsv", dtype=None, names=True)

In [467]:
#Translate longitude and latitude to Climate use binary search to look for num. coordinates
Clim = []
for i in range(0, len(lang_df)):
    s = 0
    if rco(lang_df['longitude'][i]) == -179.75:
        s = 0
    else:
        #binary search in first columns
        long = rco(lang_df['longitude'][i]) - 0.5
        s = binary(koppen['longitude'],long)
    
    if s==-1:
            Clim.append(str("NaN"))
            print(i)
    else:
        j=s
        lat = rco(lang_df['latitude'][i])
        lon = rco(lang_df['longitude'][i])
        
        while bool(lat == koppen['latitude'][j] and lon== koppen['longitude'][j])==False and bool(koppen['longitude'][j]< lon+1)==True:
            j=j+1
            if j== len(koppen['latitude'])-1:
                break
        
        if rco(lang_df['latitude'][i])== koppen['latitude'][j] and rco(lang_df['longitude'][i])==koppen['longitude'][j]:
            Clim.append(str(koppen['p1901_2010'][j]))
        else:
            Clim.append("NaN")
                                                                       
        

In [598]:
k = np.asarray(Clim)
#lang_df["Climate"] = k
# drop languages that where unale to be classified with Koppen

In [548]:
#reducing rows to those containing features with all values
df1 = lang_df[lang_df['1A_Consonant_Inventories'].notnull() & lang_df['2A_Vowel_Quality_Inventories'].notnull() & lang_df['3A_Consonant-Vowel_Ratio'].notnull() & lang_df['4A_Voicing_in_Plosives_and_Fricatives'].notnull()& lang_df['13A_Tone'].notnull() 
& lang_df['18A_Absence_of_Common_Consonants'].notnull() & lang_df['7A_Glottalized_Consonants'].notnull() & lang_df['8A_Lateral_Consonants'].notnull() & lang_df['9A_The_Velar_Nasal'].notnull()& lang_df['10A_Vowel_Nasalization'].notnull()]



In [ ]:
#Pre-processing  - features into numeric values

f1 = df1['1A_Consonant_Inventories'].str[0]
f1 = pd.to_numeric(f1)
f2 = df1['2A_Vowel_Quality_Inventories'].str[0]
f2 = pd.to_numeric(f2)
f3 = df1['3A_Consonant-Vowel_Ratio'].str[0]
f3 = pd.to_numeric(f3)
f4 = df1['4A_Voicing_in_Plosives_and_Fricatives'].str[0]
f4 = pd.to_numeric(f4)
f5 = df1['13A_Tone'].str[0]
f5 = pd.to_numeric(f5)
                   
f6 = df1['18A_Absence_of_Common_Consonants'].str[0]
f6 = pd.to_numeric(f6)
f7 = df1['7A_Glottalized_Consonants'].str[0]
f7 = pd.to_numeric(f7)
f8 = df1['8A_Lateral_Consonants'].str[0]
f8 = pd.to_numeric(f8)
f9 = df1['9A_The_Velar_Nasal'].str[0]
f9 = pd.to_numeric(f9)
f10 = df1['10A_Vowel_Nasalization'].str[0]
f10 = pd.to_numeric(f10)

#convert all series to list data structures
f1 = pd.Series.tolist(f1)
f2 = pd.Series.tolist(f2)
f3 = pd.Series.tolist(f3)
f4 = pd.Series.tolist(f4)
f5 = pd.Series.tolist(f5)
f6 = pd.Series.tolist(f6)
f7 = pd.Series.tolist(f7)
f8 = pd.Series.tolist(f8)
f9 = pd.Series.tolist(f9)
f10 = pd.Series.tolist(f10)



In [595]:
#turn series into an array of samples, where each list consist of different types of languages
X = []
f1 = pd.Series.tolist(f1)
for i in range(0, len(f1)):
    X.append([])
for i in range(0, len(f1)):
    X[i].append(f1[i])
    X[i].append(f2[i])
    X[i].append(f3[i])
    X[i].append(f4[i])
    X[i].append(f5[i])
    X[i].append(f6[i])
    X[i].append(f7[i])
    X[i].append(f8[i])
    X[i].append(f9[i])
    X[i].append(f10[i])

In [599]:
#Convert koppen classification columns into numerical category by first letter
Y = pd.Series.tolist(df1['Climate'])
for i in range(0,len(Y)):
    if Y[i][2] == 'A':
        Y[i] = 1
    elif Y[i][2] == 'B':
        Y[i] = 2
    elif Y[i][2] == 'C':
        Y[i] = 3
    elif Y[i][2] == 'D':
        Y[i] = 4
    elif Y[i][2] == 'E':
        Y[i] = 5

In [604]:
#run SVC
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer

classif = OneVsRestClassifier(estimator=SVC(random_state=0))

pred = classif.fit(X, Y).predict(X)


In [609]:
#confusion matrix for SVC
y_actu = pd.Series(Y, name='Actual')
y_pred = pd.Series(pred, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)
df_confusion

Predicted,1,2,3,4,5
Actual,,,,,
1,43,1,1,0,0
2,12,10,5,0,0
3,7,2,22,3,0
4,6,0,4,8,0
5,0,0,0,0,4


In [610]:
#Normalized confusion matrix
df_conf_norm = df_confusion / df_confusion.sum(axis=1)
df_conf_norm

Predicted,1,2,3,4,5
Actual,,,,,
1,0.955556,0.037037,0.029412,0.000000,0.0
2,0.266667,0.370370,0.147059,0.000000,0.0
3,0.155556,0.074074,0.647059,0.166667,0.0
4,0.133333,0.000000,0.117647,0.444444,0.0
5,0.000000,0.000000,0.000000,0.000000,1.0


In [619]:
#Simple accuracy measurement
c=0
for i in range(0, len(Y)):
    if Y[i]== pred[i]:
        c=c+1
c/len(Y)
    

0.6796875

In [622]:
#Cohen's Kappa Score
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(y_actu, y_pred)

0.55033844572016111